<a href="https://colab.research.google.com/github/tarikbrt2/rock-paper-scissors/blob/main/rock_paper_scissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading dataset

In [ ]:
!curl -L "https://public.roboflow.com/ds/pX4cJ7DwBg?key=dmRCWFeb9b" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

#Training our model

In [ ]:
import cv2
import numpy as np
from keras.optimizers import Adam
import tensorflow as tf
import os

IMG_PATH = 'train'

# Classes for labels
CLASSES = {
    "rock": 0,
    "paper": 1,
    "scissors": 2,
    "none": 3
}
# Number of classes for last layer
NUM_CLASSES = len(CLASSES)


def mapper(val):
    return CLASSES[val]


def crete_model():
    model = Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation="relu", input_shape=(300,300,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
    ])
    return model


# function for loading dataset
def load_data(train_path):
  dataset = []
  for directory in os.listdir(train_path):
      path = os.path.join(train_path, directory)
      if not os.path.isdir(path):
          continue
      for item in os.listdir(path):
          # to make sure no hidden files get in our way
          if item.startswith("."):
              continue
          img = cv2.imread(os.path.join(path, item))
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          img = cv2.resize(img, (300, 300))
          dataset.append([img, directory])
  return dataset

# loading dataset
dataset = load_data(IMG_PATH)

# splitting dataset into data and labels
data, labels = zip(*dataset)

# using mapper function to encode proper label into integer
labels = list(map(mapper, labels))

# one hot encode the labels
labels = np_utils.to_categorical(labels)

# creating model with our create_model() function 
model = create_model()
model.compile(
    optimizer=Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# start training with 10 epochs and defult batch_size
model.fit(np.array(data), np.array(labels), epochs=10)

# save the model for making predictions
model.save("rock-paper-scissors-model.h5")

Epoch 1/10
79/79 [==============================] - 203s 3s/step - loss: 166.1727 - accuracy: 0.5274
Epoch 2/10
79/79 [==============================] - 202s 3s/step - loss: 0.0891 - accuracy: 0.9855
Epoch 3/10
79/79 [==============================] - 202s 3s/step - loss: 0.0796 - accuracy: 0.9849
Epoch 4/10
79/79 [==============================] - 202s 3s/step - loss: 1.1598e-06 - accuracy: 1.0000
Epoch 5/10
79/79 [==============================] - 201s 3s/step - loss: 1.0282e-06 - accuracy: 1.0000
Epoch 6/10
79/79 [==============================] - 201s 3s/step - loss: 1.7720e-06 - accuracy: 1.0000
Epoch 7/10
79/79 [==============================] - 202s 3s/step - loss: 2.6646e-07 - accuracy: 1.0000
Epoch 8/10
79/79 [==============================] - 201s 3s/step - loss: 1.6979e-06 - accuracy: 1.0000
Epoch 9/10
79/79 [==============================] - 201s 3s/step - loss: 2.7368e-07 - accuracy: 1.0000
Epoch 10/10
79/79 [==============================] - 203s 3s/step - loss: 1.9685e-0

#Loding our model and making prediction

In [ ]:
from keras.models import load_model
import cv2
import numpy as np
import sys
import os

# rev class for decoding integers into proper class name
REV_CLASS_MAP = {
    0: "rock",
    1: "paper",
    2: "scissors",
    3: "none"
}


def mapper(val):
    return REV_CLASS_MAP[val]

model = load_model("rock-paper-scissors-model.h5")

TEST_PATH = "test"

# function for making and printing predictions
def make_predictions(folder_path):
  for directory in os.listdir(folder_path):
    path = os.path.join(folder_path, directory)
    if not os.path.isdir(path):
        continue
    for item in os.listdir(path):
      # loading image from test path
      img = cv2.imread(os.path.join(path, item))
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img = cv2.resize(img, (300, 300))

      # converting img into np.array and making predict
      pred = model.predict(np.array([img]))
      print(pred[0])
      move_code = np.argmax(pred[0])
      move_name = mapper(move_code)
      print("Predicted: {}\n\
      Actual: {}\n\
      ---------------------------".format(move_name, os.path.join(path, item)))

# making predictions for images inside test data
make_predictions(TEST_PATH)

[2.8575944e-15 1.0000000e+00 4.1400011e-08]
Predicted: paper
      Actual: test/paper/paper8_png.rf.452a40f1ed9e3be1e2164f14d6106f33.jpg
      ---------------------------
[4.5378182e-17 1.0000000e+00 3.6180371e-13]
Predicted: paper
      Actual: test/paper/paper6_png.rf.cd93026d24415813c3848b393e241e08.jpg
      ---------------------------
[1.0175756e-20 1.0000000e+00 4.4283505e-12]
Predicted: paper
      Actual: test/paper/paper2_png.rf.e48a2c38ee5c364493b26144d0d1b331.jpg
      ---------------------------
[1.5825921e-03 9.9841738e-01 5.5943493e-34]
Predicted: paper
      Actual: test/paper/paper1_png.rf.6771d0803db4a4cb6e5e96339f785b9d.jpg
      ---------------------------
[7.0245527e-02 9.2975444e-01 3.1244670e-20]
Predicted: paper
      Actual: test/paper/paper9_png.rf.db6fe38576ba4d5db891386b65e07cb1.jpg
      ---------------------------
[1.2091150e-05 9.9998796e-01 4.5006028e-14]
Predicted: paper
      Actual: test/paper/paper4_png.rf.8206205dc26a062ddd14c16fb7bc54d4.jpg
      --